# CS246 - Colab 2
## Frequent Pattern Mining in Spark

In [1]:
from IPython.display import Image
print("Colab 2 Mascot")
Image(url='https://cdn.dribbble.com/users/222579/screenshots/1654898/stubby-ben-rex-roll.gif',width=150)

Colab 2 Mascot


### Setup

Let's set up Spark on your Colab environment.  Run the cell below!

In [2]:
!pip install pyspark
!pip install -U -q PyDrive
!apt install openjdk-8-jdk-headless -qq
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"

openjdk-8-jdk-headless is already the newest version (8u432-ga~us1-0ubuntu2~22.04).
0 upgraded, 0 newly installed, 0 to remove and 49 not upgraded.


In [3]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


Now we authenticate a Google Drive client to download the file we will be processing in our Spark job.

**Make sure to follow the interactive instructions.**

In [4]:
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

# Authenticate and create the PyDrive client
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

In [5]:
id='1dhi1F78ssqR8gE6U-AgB80ZW7V_9snX4'
downloaded = drive.CreateFile({'id': id})
downloaded.GetContentFile('products.csv')

id='1KZBNEaIyMTcsRV817us6uLZgm-Mii8oU'
downloaded = drive.CreateFile({'id': id})
downloaded.GetContentFile('order_products__train.csv')

If you executed the cells above, you should be able to see the dataset we will need for this Colab under the "Files" tab on the left panel.

In [6]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

import pyspark
from pyspark.sql import *
from pyspark.sql.functions import *
from pyspark import SparkContext, SparkConf

Let's initialize the Spark context.

In [7]:
# create the session
conf = SparkConf().set("spark.ui.port", "4050")

# create the context
sc = pyspark.SparkContext(conf=conf)
spark = SparkSession.builder.getOrCreate()

### Your task

If you run successfully the setup stage, you are ready to work with the **3 Million Instacart Orders** dataset. In case you want to read more about it, check the [official Instacart blog post](https://tech.instacart.com/3-million-instacart-orders-open-sourced-d40d29ead6f2) about it, a concise [schema description](https://gist.github.com/jeremystan/c3b39d947d9b88b3ccff3147dbcf6c6b) of the dataset, and the [download page](https://www.instacart.com/datasets/grocery-shopping-2017).

In this Colab, we will be working only with a small training dataset (~131K orders) to perform fast Frequent Pattern Mining with the FP-Growth algorithm.

In [8]:
products = spark.read.csv('products.csv', header=True, inferSchema=True)
orders = spark.read.csv('order_products__train.csv', header=True, inferSchema=True)

In [9]:
products.printSchema()

root
 |-- product_id: integer (nullable = true)
 |-- product_name: string (nullable = true)
 |-- aisle_id: string (nullable = true)
 |-- department_id: string (nullable = true)



In [10]:
orders.printSchema()

root
 |-- order_id: integer (nullable = true)
 |-- product_id: integer (nullable = true)
 |-- add_to_cart_order: integer (nullable = true)
 |-- reordered: integer (nullable = true)



Use the Spark Dataframe API to join 'products' and 'orders', so that you will be able to see the product names in each transaction (and not only their ids).  Then, group by the orders by 'order_id' to obtain one row per basket (i.e., set of products purchased together by one customer).

In [11]:
''' 2 lines of code expected '''
# YOUR CODE HERE
df = products.join(orders, products.product_id == orders.product_id).groupby("order_id").agg(collect_list("product_name").alias("list"))

In this Colab we will explore [MLlib](https://spark.apache.org/mllib/), Apache Spark's scalable machine learning library. Specifically, you can use its implementation of the [FP-Growth](https://spark.apache.org/docs/latest/ml-frequent-pattern-mining.html#fp-growth) algorithm to perform efficiently Frequent Pattern Mining in Spark.
Use the Python example in the documentation, and train a model with

```minSupport=0.01``` and ```minConfidence=0.5```



In [12]:
df.take(3)

[Row(order_id=1, list=['Bulgarian Yogurt', 'Organic 4% Milk Fat Whole Milk Cottage Cheese', 'Organic Celery Hearts', 'Cucumber Kirby', 'Lightly Smoked Sardines in Olive Oil', 'Bag of Organic Bananas', 'Organic Hass Avocado', 'Organic Whole String Cheese']),
 Row(order_id=96, list=['Roasted Turkey', 'Organic Cucumber', 'Organic Grape Tomatoes', 'Organic Pomegranate Kernels', 'Organic Raspberries', 'Organic Whole Strawberries', 'Organic Blueberries']),
 Row(order_id=112, list=['Fresh Cauliflower', 'I Heart Baby Kale', 'Sea Salt Baked Potato Chips', 'Marinara Pasta Sauce', 'Organic Hass Avocado', 'Organic Lemon', 'Coconut Water Kefir', 'Premium Epsom Salt', 'Umcka Elderberry Intensive Cold + Flu Berry Flavor', 'Hickory Honey Barbeque Baked Potato Chips', 'Baked Sea Salt & Vinegar Potato Chips'])]

In [13]:
''' 3 lines of code expected '''
# YOUR CODE HERE
from pyspark.ml.fpm import FPGrowth
fpGrowth = FPGrowth(itemsCol="list", minSupport=0.01, minConfidence=0.5)
model = fpGrowth.fit(df)

Compute how many frequent itemsets and association rules were generated by running FP-growth alongside visalizing top frequent itemsets and association rules.


In [14]:
''' 5 lines of code in total expected but can differ based on your style. for sub-parts of the question, creating different cells of code would be recommended.'''
# YOUR CODE HERE
model.freqItemsets.count()

120

In [15]:
model.freqItemsets.orderBy(desc("freq")).take(20)

[Row(items=['Banana'], freq=18726),
 Row(items=['Bag of Organic Bananas'], freq=15480),
 Row(items=['Organic Strawberries'], freq=10894),
 Row(items=['Organic Baby Spinach'], freq=9784),
 Row(items=['Large Lemon'], freq=8135),
 Row(items=['Organic Avocado'], freq=7409),
 Row(items=['Organic Hass Avocado'], freq=7293),
 Row(items=['Strawberries'], freq=6494),
 Row(items=['Limes'], freq=6033),
 Row(items=['Organic Raspberries'], freq=5546),
 Row(items=['Organic Blueberries'], freq=4966),
 Row(items=['Organic Whole Milk'], freq=4908),
 Row(items=['Organic Cucumber'], freq=4613),
 Row(items=['Organic Zucchini'], freq=4589),
 Row(items=['Organic Yellow Onion'], freq=4290),
 Row(items=['Organic Garlic'], freq=4158),
 Row(items=['Seedless Red Grapes'], freq=4059),
 Row(items=['Asparagus'], freq=3868),
 Row(items=['Organic Grape Tomatoes'], freq=3823),
 Row(items=['Organic Red Onion'], freq=3818)]

In [21]:
model.associationRules.count()

11

In [16]:
model.associationRules.show()

+----------+----------+----------+----+-------+
|antecedent|consequent|confidence|lift|support|
+----------+----------+----------+----+-------+
+----------+----------+----------+----+-------+



Now retrain the FP-growth model changing only
```minsupport=0.001```
and compute how many frequent itemsets and association rules were generated.


In [17]:
''' 5 lines of code in total expected but can differ based on your style. for sub-parts of the question, creating different cells of code would be recommended.'''
# YOUR CODE HERE
fpGrowth = FPGrowth(itemsCol="list", minSupport=0.001, minConfidence=0.5)
model = fpGrowth.fit(df)

In [18]:
model.freqItemsets.count()

4444

In [19]:
model.associationRules.count()

11

To conclude, go to Gradescope and read the remaining questions. We will ask you to inspect the resulting dataframes, and report a few results.



In [20]:
# YOUR CODE HERE
model.associationRules.take(20)

[Row(antecedent=['Organic Whole String Cheese', 'Organic Hass Avocado'], consequent=['Bag of Organic Bananas'], confidence=0.5314685314685315, lift=4.504745125675359, support=0.0011584571180330617),
 Row(antecedent=['Organic Broccoli', 'Organic Hass Avocado'], consequent=['Bag of Organic Bananas'], confidence=0.5048231511254019, lift=4.278897986822536, support=0.001196564260073623),
 Row(antecedent=['Organic Navel Orange', 'Organic Raspberries'], consequent=['Bag of Organic Bananas'], confidence=0.5412186379928315, lift=4.587387356098284, support=0.0011508356896249496),
 Row(antecedent=['Organic Kiwi', 'Organic Hass Avocado'], consequent=['Bag of Organic Bananas'], confidence=0.5459770114942529, lift=4.627719489738336, support=0.001448071397541327),
 Row(antecedent=["Organic D'Anjou Pears", 'Organic Hass Avocado'], consequent=['Bag of Organic Bananas'], confidence=0.5170454545454546, lift=4.3824946411792345, support=0.0013870999702764292),
 Row(antecedent=['Organic Raspberries', 'Organ

Once you obtained the desired results, **head over to Gradescope and submit your solution for this Colab**! As a sanity check, for `minSupport=0.01` you should obtain `freq([Limes]) = 6033`